# Assignment: Decision trees and random forests

### Objectives

The objectives of this assignment are:

to learn to use decision trees to build a transparent, human-readable predictive model.
to learn to use random forests for improved predictive performance.
Setup
In the assignment, use the Phishing dataset that is available at the UCI Machine Learning Repository: Phishing Websites Data Set. The target variable Result indicates whether a website is a phishing site or not.

Note: As the interpretation of the -1’s and 1’s in the Result column seems to be missing from the document, it may be helpful to know that a ‘1‘ corresponds to a phishing site and a ‘-1’ to a legitimate site.

Hint: Click the Download in Python button for quick access to the dataset. In addition, click the Download button to gain access to the dataset description.

Task
Your goal is to find out whether it is possible to reliably predict whether a website is a phishing site or not based on the easily obtainable information about the website. Based on the outcome, it may become possible to construct an automated system that warns users when they are about to visit a phishing website.

Part 1: Decision tree
Your initial goal is to construct a small yet useful decision tree that predicts whether a website is a phishing site or not.

The outcome should contain the following:

An image of the final decision tree.
Evaluation metrics for the decision tree.
Written instructions for an internet analyst to manually make the decision of whether the website is likely to be a phishing site or not. The instructions must match one-to-one with your decision tree, and be written in a way that is understandable to an engineer who is aware of the basics of internet technologies.
Part 2: Random forest
As the ultimate goal is to build an automated system, you don't have to stick to a single, relatively simple decision tree.

Try to tweak the performance of the decision tree by replacing it with a random forest. You may also try to tune the hyperparameters of the random forest to improve the performance.

Be sure to include the validation results in your report.

In real life, when you tune the hyperparameters based on the validation results, you should have yet another data set that is not used for tuning the hyperparameters, but applied only once after the tuning of the hyperparameters to obtain the final performance estimate of the tuned model. That is, there should be three sets: training, validation, and test sets. On this course, you may skip the need for the third set.

Deliverables
Submit a GitHub permalink that points to the Jupyter notebook as instructed in Oma. The submitted notebook must contain the problem analysis written in accordance with the CRISP-DM process model, complete with Markdown blocks and comments that clearly explain what has been done.

In [27]:
from sklearn.model_selection import train_test_split
from ucimlrepo import fetch_ucirepo
import pandas as pd
import matplotlib.pyplot as plt

# fetch dataset
phishing_websites = fetch_ucirepo(id=327)

# data (as pandas dataframes)
X = phishing_websites.data.features
y = phishing_websites.data.targets

# Split data for validation purpose
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=15)
y

,result
0,-1
1,-1
2,-1
3,-1
4,1
...,...
11050,1
11051,-1
11052,-1
11053,-1


# Building a decision tree

In [28]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=52, ccp_alpha=0.004, max_depth=4)
model.fit(X_train, y_train)
print(model.classes_)

[-1  1]


# Decision tree

In [29]:
from sklearn.tree import plot_tree

fig = plt.figure(figsize = (12, 12))
plot_tree(model, feature_names = X.keys(), class_names = ['legitimate', 'phishing'])
plt.show()

Check for SSL Connection:
- If there is an SSL connection between the client (usually a browser) and the server:
- Check if there are anchor elements hiding URLs. If yes, the site is a phishing site.
- If not, check the web traffic. If there is web traffic, the site is most likely legitimate.
- If there is no SSL connection:
- Check if there is a suspicious link hiding in an anchor element (<a> tag). If yes, the site is most likely a phishing site.
- If not, check if the number of links in tags is less than or equal to -0.5. If yes, the site is a phishing site.
- If not, check if there are suspicious prefixes or suffixes in the URL. If yes, the site is a phishing site. If no, the site is most likely legitimate.

# Evaluation metrics

              precision    recall  f1-score   support

  Legitimate       0.90      0.90      0.90      1457
    Phishing       0.92      0.92      0.92      1860

    accuracy                           0.91      3317
   macro avg       0.91      0.91      0.91      3317
weighted avg       0.91      0.91      0.91      3317



In [30]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

predictions = model.predict(X_test)
cm = confusion_matrix(y_test, predictions)
report = classification_report(y_test, predictions, target_names=['Legitimate', 'Phishing'])
print(report)
cmd = ConfusionMatrixDisplay(cm, display_labels=['legitimate', 'phishing'])
cmd.plot(cmap='Blues')


# Random forest build

In [31]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# Create a random forest classifier
modelRand = RandomForestClassifier(n_estimators=100, bootstrap=True, max_samples=0.7, max_features=0.75, random_state=123, ccp_alpha=0.009)

# Carry out 10-fold cross-validation
cv_scores = cross_val_score(modelRand, X, y.values.ravel(), cv=10)

# Show results
print(f'Fold-specific accuracies: {cv_scores}')
print(f'Overall accuracy (avg. of folds): {cv_scores.mean():.4f}')


Fold-specific accuracies: [0.90506329 0.90325497 0.91048825 0.91229656 0.91862568 0.92760181
 0.91493213 0.90497738 0.9239819  0.92217195]
Overall accuracy (avg. of folds): 0.9143


# Random forest trees

In [32]:
# Number of trees to plot
n_trees_to_plot = 5

modelRand.fit(X, y.values.ravel())


# Create a figure to hold the plots
plt.figure(figsize=(20, 10))

for i in range(n_trees_to_plot):
    plt.subplot(1, n_trees_to_plot, i + 1)  # 1 row and n_trees_to_plot columns
    plot_tree(modelRand.estimators_[i], filled=True,
              feature_names=phishing_websites.feature_names, class_names=phishing_websites.target_names)
    plt.title(f'Tree {i + 1}')

plt.tight_layout()
plt.show()

# Evaluation for random forest

              precision    recall  f1-score   support

  Legitimate       0.90      0.90      0.90      1457
    Phishing       0.92      0.92      0.92      1860

    accuracy                           0.91      3317
   macro avg       0.91      0.91      0.91      3317
weighted avg       0.91      0.91      0.91      3317



In [33]:
predictionsRandom = modelRand.predict(X_test)
cm = confusion_matrix(y_test, predictions)

cmd = ConfusionMatrixDisplay(cm, display_labels=['legitimate', 'phishing'])
report = classification_report(y_test, predictionsRandom, target_names=['Legitimate', 'Phishing'])
print(report)
cmd.plot(cmap='Blues')

#